# Oppening statement

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress
from pandas import Series
from pandas import DataFrame as DF

from core.pandas_utils import floor_to
from core.plt_utils import plt_3d_df
from transform.watea.soh_estimation import get_processed_cluster, get_soh_per_charges

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
processed_cluster = get_processed_cluster()
charges = get_soh_per_charges()

In [ ]:
soh_per_vehicle = charges.groupby('id').agg({
    "soh": "mean",
    "odometer": "max",
}).reset_index(drop=False)

## soh results

In [ ]:
px.scatter(
    charges.assign(soh=charges["soh"].sub(2.5).clip(90, 100)),
    x="odometer",
    y="soh",
    color="id",
    trendline="ols",
    trendline_scope="overall",
)

In [ ]:
px.scatter(
    soh_per_vehicle.assign(soh=soh_per_vehicle["soh"].sub(1.5)),
    x="odometer",
    y="soh",
    # color="id",
    trendline="ols",
    trendline_scope="overall",
)

## Energy consumption over soc and temeperature

In [ ]:
processed_cluster["floored_temperature"] = floor_to(processed_cluster["temperature"], 5)
dist_to_plot = (
    processed_cluster
    .query("temperature < 30 & temperature > 0")
    .groupby(["soc", "floored_temperature"])[["energy_added"]]
    .median()
    .reset_index()
    .sort_values(by=["floored_temperature", "soc"], ascending=[False, True])
)
fig = px.line(
    dist_to_plot,
    x="soc",
    y="energy_added",
    color="floored_temperature",
    color_discrete_sequence=px.colors.sequential.Rainbow,
)

fig.show()

In [ ]:
# 👉 Slide type of this cell: Fragment

px.box(
    dist_to_plot,
    "floored_temperature",
    "energy_added",
    color="floored_temperature",
    color_discrete_sequence=px.colors.sequential.Rainbow,
).show()

In [ ]:
# ! jupyter nbconvert watea_presentation.ipynb --to slides --no-prompt --TagRemovePreprocessor.remove_input_tags={\"to_remove\"} --post serve  --no-input